A partir de una tabla de datos ya procesada para utilizar con los modelos teoricos, 
en este nb buscamos optimizar

In [1]:
## seteamos los parametros del notebook
%autosave 180

import pandas as pd; 
import numpy as np; 
import os;
import random;
import time
import matplotlib
%matplotlib inline
import sys

np.random.seed(2015)
rootdir=os.getcwd()
rootdir+= "/datasets"
os.chdir(rootdir)

Autosaving every 180 seconds


In [2]:
#from unbalanced_dataset import UnderSampler, NearMiss, CondensedNearestNeighbour, OneSidedSelection,\
#NeighbourhoodCleaningRule, TomekLinks, ClusterCentroids, OverSampler, SMOTE,\
#SMOTETomek, SMOTEENN, EasyEnsemble, BalanceCascade

In [5]:
ls ../

AntennaVulnAggregation.ipynb
Antennas department and epidemic.ipynb
Antennas plot??.ipynb
DevicesTesting.csv.gz
Exploracion de geo voice dataset.ipynb
Past_label_extraction.ipynb
Past_label_extraction.py
README.md
SimpleFormat*
SimpleFormat data structure exploration.ipynb
Sum_Links attribute extraction.ipynb
User_antenna mapping.ipynb
User_antenna mapping.py
antennas_mexico.csv
backup_output1.txt.gz
build/
clustering/
dataDescribe.ipynb
datasets/
datosgeo/
graphLabClassify.ipynb
mapas/
mapas-mex.tar.gz
mexico_ant2ant_aggregation.txt.gz
mexico_ant2vuln_aggregation.txt
mexico_ant2vuln_aggregation_backup.txt
minutasreuniones.synctex.gz
minutasreuniones.tex
mobilityClassifiers.ipynb
output1.txt.gz
output2.txt.gz
output_sum_links.txt.gz
past_labels.txt.gz
preoutput_sum_links.txt
preoutput_sum_links.txt.gz
scikit-learn/
scripts-antennas-preprocessing/
simple_format.sh*
sklearn.tar.gz
sklearnClassifiers.ipynb
surrogated_antennas.txt.gz
surrogated_antennas_complete.txt
surrogated_antennas_epi

In [7]:
ls *.gz

ground_truth.csv.gz  sf_output.txt.gz  sl_output.txt.gz


In [4]:
!zcat ground_truth.csv.gz| head

USER|ANTENNA_ID|EPIDEMIC|EXPOSED
FFFF51FDDD55A2E24D074DE30C4798E9|470|0|0
FFFC3133FAEF588B9FA75487C864F774|1036|0|1
FFFB2F18F4CCF253C06B581EA518A7A5|497|1|1
FFFB2C0FEE677EE733DDD85F11C5A14C|214|0|1
FFFB27D77413E57C38D14D876C3DE8CF|2946|0|1
FFFA0736F66FB09200CE1D13E081A946|15|1|1
FFF976C823C8ACDE846E68DF75F4D698|792|1|1
FFF8ECD2969C8CD09D164D67F9BA206A|884|0|1
FFF8373FC0468150A4B6460E6F1A3768|2297|1|1

gzip: stdout: Broken pipe


In [8]:
file_sf = "sf_output.txt.gz"
file_sl = "sl_output.txt.gz"
file_gt = "ground_truth.csv.gz"

sf_table = pd.read_csv(file_sf,sep = ",",
                   header =0,
                     index_col =0)
sl_table = pd.read_csv(file_sl,sep = "|",
                   header =0,
                     index_col =0)
gt_table = pd.read_csv(file_gt,sep = "|",
                   header =0,
                     index_col =0)

#agrego la etiqueta "_gt" a las columnas del ground_truth
gt_table.columns = [col+"_gt" for col in gt_table.columns]

In [9]:
sf_table.shape, sl_table.shape, gt_table.shape

((3667451, 23), (2449242, 15), (2000285, 3))

In [10]:
#checking table consistency
np.in1d(gt_table.index.values,sf_table.index.values).sum()

2000285

In [11]:
np.in1d(gt_table.index.values,sl_table.index.values).sum()

2000285

In [13]:
sf_table.head()

,EXPOSED,ANTENNA_ID_0,COUNT_0,ANTENNA_ID_1,COUNT_1,ANTENNA_ID_2,COUNT_2,ANTENNA_ID_3,COUNT_3,ANTENNA_ID_4,...,ANTENNA_ID_6,COUNT_6,ANTENNA_ID_7,COUNT_7,ANTENNA_ID_8,COUNT_8,ANTENNA_ID_9,COUNT_9,EPIDEMIC,MOBILITY_DIAMETER
USER,,,,,,,,,,,,,,,,,,,,,
FFFF78CB080636632B2D1DE7A7BFAB03,False,3415,21,2969,5,1320,3,2471,3,1115,...,-1,65535,-1,65535,-1,65535,-1,65535,False,1063.140535
FFFF51FDDD55A2E24D074DE30C4798E9,True,1207,15,1171,14,1820,9,583,8,2026,...,1831,4,1625,2,401,1,825,1,False,520.065959
FFFE1EDDA25369CBC467536FA6A787FD,True,1847,22,1753,14,1422,11,1028,10,765,...,862,8,2463,7,1416,6,871,5,True,989.597851
FFFD0A912A959CDFCFFAB2F93BFBA435,True,1407,18,1702,8,1613,6,1704,5,1394,...,818,2,958,2,726,1,848,1,False,490.585382
FFFCB235D6D917DDF350F5B11CCB0EE8,True,17,74,174,33,1123,4,1178,4,173,...,503,1,1039,1,1228,1,1327,1,True,1613.225606


In [7]:
sl_table.head()

,CallsWeekDaylight,CallsWeekDaylight_EPI,CallsWeekNight,CallsWeekNight_EPI,CallsWeekend,CallsWeekend_EPI,TimeWeekDaylight,TimeWeekDaylight_EPI,TimeWeekNight,TimeWeekNight_EPI,TimeWeekend,TimeWeekend_EPI,TOTAL_USERS,EPI_USERS,EXP_USERS
LineKeyOrigin,,,,,,,,,,,,,,,
000004F96010462FA7A71C7784E27476,0,0,0,0,0,1,0,0,0,0,0,34,1,0,1
000007EEE7895E17937CFF86379C8E17,0,2,0,0,0,0,0,144,0,0,0,0,1,1,1
00000837B868F926F67BD440A0845C1D,1,0,2,4,3,0,47,0,74,86,109,0,3,0,2
00000C5D3BFBEBD919F6F68386C3DD5E,4,0,0,0,0,0,281,0,0,0,0,0,1,0,1
0000129F4191FFD0963774D2307C7510,2,0,0,0,0,0,66,0,0,0,0,0,2,0,2


In [25]:
gt_table.head()

,ANTENNA_ID_gt,EPIDEMIC_gt,EXPOSED_gt
USER,,,
FFFF51FDDD55A2E24D074DE30C4798E9,470,0,0
FFFC3133FAEF588B9FA75487C864F774,1036,0,1
FFFB2F18F4CCF253C06B581EA518A7A5,497,1,1
FFFB2C0FEE677EE733DDD85F11C5A14C,214,0,1
FFFB27D77413E57C38D14D876C3DE8CF,2946,0,1


In [12]:
# para poder usar el estimador MultionmialNB, paso todos los valores negativos en las antenna_ids de sf_table a 0

for col in [col for col in sf_table.columns.values if "_ID" in col]:
    sf_table[col] = sf_table[col].replace(-1,0)

In [43]:
X_train = pd.concat([gt_table,sf_table, sl_table], axis=1, join='inner')
y_train = X_train[[col for col in X_train.columns if "_gt" in col]].copy()
X_train = X_train[[col for col in X_train.columns if not ("_gt" in col)]]

## Choosing different fitting objectives
We have enough information to attack the following questions:
* Predict if user A used to be exposed.
* Predict if user A used to be epidemic.
* Predict if user A used to have Antenna N as a base one. (antenna number has to be given and given that we have more than 4500 antenna, classes will be EXTREMELY unbalanced in this scenario )

In [10]:
X_train.head()

,ANTENNA_ID_gt,EPIDEMIC_gt,EXPOSED_gt,EXPOSED,ANTENNA_ID_0,COUNT_0,ANTENNA_ID_1,COUNT_1,ANTENNA_ID_2,COUNT_2,...,CallsWeekend_EPI,TimeWeekDaylight,TimeWeekDaylight_EPI,TimeWeekNight,TimeWeekNight_EPI,TimeWeekend,TimeWeekend_EPI,TOTAL_USERS,EPI_USERS,EXP_USERS
000004F96010462FA7A71C7784E27476,3471,1,1,True,2937,36,1787,22,3314,19,...,1,0,0,0,0,0,34,1,0,1
000007EEE7895E17937CFF86379C8E17,952,1,1,True,625,6,33,5,1915,5,...,0,0,144,0,0,0,0,1,1,1
00000837B868F926F67BD440A0845C1D,2836,1,1,True,650,102,1955,48,1915,26,...,0,47,0,74,86,109,0,3,0,2
00000C5D3BFBEBD919F6F68386C3DD5E,3785,0,1,False,1325,3,2699,1,3227,1,...,0,281,0,0,0,0,0,1,0,1
0000129F4191FFD0963774D2307C7510,415,0,1,True,2226,18,306,12,891,11,...,0,66,0,0,0,0,0,2,0,2


In [35]:
y_table.head()

,ANTENNA_ID_gt,EPIDEMIC_gt,EXPOSED_gt
000004F96010462FA7A71C7784E27476,3471,1,1
000007EEE7895E17937CFF86379C8E17,952,1,1
00000837B868F926F67BD440A0845C1D,2836,1,1
00000C5D3BFBEBD919F6F68386C3DD5E,3785,0,1
0000129F4191FFD0963774D2307C7510,415,0,1


## Scikit estimator fit and cross-validation

In [37]:
from sklearn.naive_bayes import *
from sklearn.linear_model import *
from sklearn.ensemble import *
from sklearn.svm import *
from sklearn.neural_network import *
from sklearn.cross_validation import *
from sklearn.multiclass import OneVsRestClassifier
from sklearn.utils import *
from sklearn.preprocessing import label_binarize, scale, StandardScaler
from sklearn.metrics import *
from sklearn.ensemble import *
from sklearn.tree import *
from sklearn.grid_search import *
from sklearn.pipeline import Pipeline

/home/juan/mobility-study/scikit-learn/sklearn/cross_validation.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/juan/mobility-study/scikit-learn/sklearn/grid_search.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Create new variables
These would be more "real" variables i.e. variables that would be "humanly" explainable

In [42]:
!zcat $file_sl | head

LineKeyOrigin|CallsWeekDaylight|CallsWeekNight|CallsWeekend|TimeWeekDaylight|TimeWeekNight|TimeWeekend|CallsWeekDaylight_EPI|CallsWeekNight_EPI|CallsWeekend_EPI|TimeWeekDaylight_EPI|TimeWeekNight_EPI|TimeWeekend_EPI|TOTAL_USERS|EPI_USERS|EXP_USERS
000004F96010462FA7A71C7784E27476|0|0|1|0|0|34|0|0|0|0|0|0|1|0|1
000007EEE7895E17937CFF86379C8E17|2|0|0|144|0|0|2|0|0|144|0|0|1|1|1
00000837B868F926F67BD440A0845C1D|1|6|3|47|160|109|0|0|0|0|0|0|3|0|2
00000C5D3BFBEBD919F6F68386C3DD5E|4|0|0|281|0|0|0|0|0|0|0|0|1|0|1
0000129F4191FFD0963774D2307C7510|2|0|0|66|0|0|0|0|0|0|0|0|2|0|2
000017368A8964F8C71A9F3A1EAF6160|7|0|1|960|0|58|1|0|0|147|0|0|4|1|4
00001ADC8C5705AB4FD8197B5024C1AA|8|10|5|773|542|359|1|0|1|48|0|42|4|2|4
00001B448AE41076EB2C58ABEAF0E0C8|15|13|11|1319|736|488|4|0|4|275|0|116|2|1|2
00001C544BEBBB9109CE3552B4CDC490|0|1|0|0|36|0|0|0|0|0|0|0|1|0|1

gzip: stdout: Broken pipe


In [39]:
X_train.columns

Index([u'EXPOSED', u'ANTENNA_ID_0', u'COUNT_0', u'ANTENNA_ID_1', u'COUNT_1',
       u'ANTENNA_ID_2', u'COUNT_2', u'ANTENNA_ID_3', u'COUNT_3',
       u'ANTENNA_ID_4', u'COUNT_4', u'ANTENNA_ID_5', u'COUNT_5',
       u'ANTENNA_ID_6', u'COUNT_6', u'ANTENNA_ID_7', u'COUNT_7',
       u'ANTENNA_ID_8', u'COUNT_8', u'ANTENNA_ID_9', u'COUNT_9', u'EPIDEMIC',
       u'MOBILITY_DIAMETER', u'CallsWeekDaylight', u'CallsWeekNight',
       u'CallsWeekend', u'TimeWeekDaylight', u'TimeWeekNight', u'TimeWeekend',
       u'CallsWeekDaylight_EPI', u'CallsWeekNight_EPI', u'CallsWeekend_EPI',
       u'TimeWeekDaylight_EPI', u'TimeWeekNight_EPI', u'TimeWeekend_EPI',
       u'TOTAL_USERS', u'EPI_USERS', u'EXP_USERS'],
      dtype='object')

In [ ]:
calls_cols = [col for col in X_train.columns if "Calls" in col] 
time_cols = [col for col in X_train.columns if "Time" in col]
epi_calls_cols = [col for col in X_train.columns if "Calls" in col] 
epi_time_cols = [col for col in X_train.columns if "Time" in col and ""]

In [ ]:
# datafram test train split

# size of train set
perc =0.8
msk = np.random.rand(len(X_train)) < perc
X_val = X_train.ix[~msk]

X_train = X_train.ix[msk]

In [ ]:
#now selection for target columns

gt = "EPIDEMIC_gt"
#exp_gt = True
#ant_gt = 

y_val = y_train.loc[~msk,gt]
y_train = y_train.loc[msk,gt]

## Create new features
these would be "synthetic" variables

In [ ]:


def process_X_files(tables, scaled=False, poly2=False,train_size = 0.8):
    #como input tiene que ir una lista de tables donde la tabla donde se fitea todo viene PRIMERA, i.e. en la pos 0
    #tiene que suceder que haya consistencia entre las columnas de c/dataframe tambien
    train_table = tables[0]
    categorical_cols = [col for col in train_table.columns if not("count" in col or "time_" in col or "y" in col)]
    count_time_cols = [col for col in train_table.columns if (("count" in col or "time_" in col) and not("y" in col))]
    
    print("First dataframe is %s big" % str(train_table.shape))
    #print("Test dataframe is %s big" % str(test_table.shape))
    
    
    X_train_categorical = train_table[categorical_cols].values
    X_train_count_time = train_table[count_time_cols].values
    
    if scaled ==True :
            min_max_scaler = MinMaxScaler().fit(X_train_count_time)
            X_train_count_time = min_max_scaler.transform(X_train_count_time)
    
    #hacemos interacciones polinomiales sobre las columnas de count/time
    if poly2 == True:
        #pensar que hacer polynomial features es masomenos como agregar nˆ2 nuevas columnas
        #con lo cual tenemos que tener cuidado en no reventar la memoria, ponemor las primeras 50 columnas 
        #como tope para aplicar interacciones polinomiales

        poly2_transform = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
        max_cols = 50

        if  (X_train_count_time.shape[1] <= max_cols):
            max_cols = X_train_count_time.shape[1]
        poly2_transform.fit(X_train_count_time[:,0:max_cols])

        X_train_poly2 = poly2_transform.transform(X_train_count_time[:,0:max_cols])
        X_train_count_time = np.concatenate( ( X_train_poly2, X_train_count_time[:,max_cols:]),axis=1) 

    X_train = np.concatenate( ( X_train_count_time, X_train_categorical),axis=1)
    
    processed_tables = [X_train]
    
    #aca basicamente replicamos lo anterior pero para todo el resto de las tablas y utilizando los fits que ya tenemos

    for i in range(len(tables)):
        # skip the X_train table which comes in the first postion and will behave as our special fitting table
        if i ==0:
            continue
        print(i)
        X_val_categorical = tables[i][categorical_cols].values
        X_val_count_time = tables[i][count_time_cols].values

        if scaled ==True :
            X_val_count_time = min_max_scaler.transform(X_val_count_time)

        #hacemos interacciones polinomiales sobre las columnas de count/time
        if poly2 == True:
            #pensar que hacer polynomial features es masomenos como agregar nˆ2 nuevas columnas
            #con lo cual tenemos que tener cuidado en no reventar la memoria, ponemor las primeras 50 columnas 
            #como tope para aplicar interacciones polinomiales

            X_val_poly2 = poly2_transform.transform(X_val_count_time[:,0:max_cols])
            X_val_count_time = np.concatenate( ( X_val_poly2, X_val_count_time[:,max_cols:]),axis=1) 

        X_val = np.concatenate( ( X_val_count_time, X_val_categorical),axis=1)
        
        processed_tables = processed_tables + [X_val]
    
    for i in range(len(processed_tables)):    
        print("Table {0} shape is {1}".format( i,str(processed_tables[i].shape)))
#        print("Table %s categorical shape is %s" % str(X_val_categorical.shape))
#        print("Val non-categorical shape is %s" % str(X_val_count_time.shape))
#        print("Val categorical shape is %s" % str(X_val_categorical.shape))
    
    return processed_tables

In [14]:
## seteo los generalizadores de CV
##genero los estimadores
W = np.array([15 if i == 1 else 1 for i in y])
#model = GradientBoostingClassifier()

gradboost = GradientBoostingClassifier( loss = 'deviance',n_estimators=20, 
                                       max_depth = 20
                                 )
svc = LinearSVC(C=2.0, tol = 1e-4,
                class_weight = 'balanced'
               )

svc2 = SVC(C=2.0, kernel = 'rbf' ,decision_function_shape='ovr',tol = 1e-5,
                class_weight = 'balanced', max_iter = 1000
               )


# Multinomial Naive Bayes - Benchmark - Cross Validation
mnb es bastante rapido y eficiente para rapidamente resultados en problemas de clasificacion supervisada

In [47]:
start_time = time.time()
param_grid = {'alpha':[0,0.5,1.0, 1e-1,1e-2,1e1], 'fit_prior': [True,False],
             }
estimator  = MultinomialNB( )

clf =GridSearchCV(estimator, param_grid, scoring='f1', fit_params=None, n_jobs=10, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')
n_iter_search =60

#how many parameters to randomly search for
n_iter_search = 45

#clf = RandomizedSearchCV(estimator, param_distributions=param_grid,
#                                   n_iter=n_iter_search, n_jobs =-1, verbose=3)


#X = X_train_categorical
clf.fit(X_train,y_train)


#random_search.fit(X_train,y_train)
elapsed_time =  time.time() - start_time

print('Random Search took %s seconds to run' % elapsed_time)


all_time =   time.time() - start_time 
print('Grid Search took %s seconds to run' % (all_time - elapsed_time))



predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['class'] = clf.predict(X_test)
predicted_proba[gt] =  y_test
predicted_proba.sort_values(by = 'prob',ascending = False, inplace = True)

cut_percentages = [10,20,30,40]
#converted = (predicted_proba['class']==1)
#me fijo todos los users que van a convertir en el futuro


start = 0
converted_dict = {}
ind_segments = []
for i in cut_percentages:
    cut_size = int(len(predicted_proba)*i/100.0)    
    bool_array = (~np.ones(len(predicted_proba), dtype=bool))
    bool_array[start: start + cut_size] = True
    ind_segments += [bool_array]
    start = start + cut_size
#    subtable = 
    #converted_dict[str(i)] = predicted_proba.loc[ind_segments[-1],'ref_hash']
    
    print(classification_report(predicted_proba.loc[ind_segments[-1],gt].values,
                            predicted_proba.loc[ind_segments[-1],'class'].values ))
    
    
print(classification_report(predicted_proba.loc[:,gt].values,
                            predicted_proba.loc[:,'class'].values ))
    
    
cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1
#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

all_time =   time.time() - start_time

#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

print('This cell took %s seconds to run' % all_time)



Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] alpha=0, fit_prior=True .........................................
[CV] ................ alpha=0, fit_prior=True, score=0.505480 -   7.6s
[CV] alpha=0, fit_prior=True .........................................
[CV] ................ alpha=0, fit_prior=True, score=0.500965 -   7.7s
[CV] alpha=0, fit_prior=True .........................................
[CV] ................ alpha=0, fit_prior=True, score=0.505311 -   7.6s
[CV] alpha=0, fit_prior=False ........................................
[CV] ............... alpha=0, fit_prior=False, score=0.505498 -   7.5s
[CV] alpha=0, fit_prior=False ........................................
[CV] ............... alpha=0, fit_prior=False, score=0.500967 -   7.5s
[CV] alpha=0, fit_prior=False ........................................
[CV] ............... alpha=0, fit_prior=False, score=0.505320 -   7.7s
[CV] alpha=0.5, fit_prior=True .......................................
[CV] ...........

[Parallel(n_jobs=10)]: Done  31 tasks       | elapsed:  4.3min
[Parallel(n_jobs=10)]: Done  36 out of  36 | elapsed:  5.0min finished



Random Search took 306.483228922 seconds to run
Grid Search took 0.000793218612671 seconds to run
             precision    recall  f1-score   support

          0       0.64      0.24      0.35     41568
          1       0.37      0.77      0.50     24441

avg / total       0.54      0.43      0.40     66009

             precision    recall  f1-score   support

          0       0.64      0.24      0.34     83362
          1       0.37      0.77      0.50     48657

avg / total       0.54      0.43      0.40    132019

             precision    recall  f1-score   support

          0       0.64      0.24      0.35    124460
          1       0.37      0.77      0.50     73568

avg / total       0.54      0.44      0.40    198028

             precision    recall  f1-score   support

          0       0.64      0.24      0.35    166086
          1       0.37      0.77      0.50     97952

avg / total       0.54      0.44      0.41    264038

             precision    recall  f1-scor

# Results
* fitting prior has better mean and almost same std
* alpha should be only > 0. But difference between positive alphas is negligible, even on different exponential orders
* 

## Logistic Regression cross validation

In [54]:

clf =  LogisticRegressionCV(n_jobs =3, class_weight= 'balanced', scoring = 'f1',cv=3)

clf.fit(X_train,y_train)


predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['class'] = clf.predict(X_test)
predicted_proba[gt] =  y_test
predicted_proba.sort_values(by = 'prob',ascending = False, inplace = True)

cut_percentages = [10,20,30,40]
#converted = (predicted_proba['class']==1)
#me fijo todos los users que van a convertir en el futuro


start = 0
converted_dict = {}
ind_segments = []
for i in cut_percentages:
    cut_size = int(len(predicted_proba)*i/100.0)    
    bool_array = (~np.ones(len(predicted_proba), dtype=bool))
    bool_array[start: start + cut_size] = True
    ind_segments += [bool_array]
    start = start + cut_size
#    subtable = 
    #converted_dict[str(i)] = predicted_proba.loc[ind_segments[-1],'ref_hash']
    
    print(classification_report(predicted_proba.loc[ind_segments[-1],gt].values,
                            predicted_proba.loc[ind_segments[-1],'class'].values ))
    
    
print(classification_report(predicted_proba.loc[:,gt].values,
                            predicted_proba.loc[:,'class'].values ))
    
    
cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1
#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

all_time =   time.time() - start_time

#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

print('This cell took %s seconds to run' % all_time)


             precision    recall  f1-score   support

          0       0.67      0.85      0.75     43545
          1       0.40      0.19      0.25     22464

avg / total       0.58      0.63      0.58     66009

             precision    recall  f1-score   support

          0       0.64      0.75      0.69     83353
          1       0.40      0.28      0.33     48666

avg / total       0.55      0.58      0.56    132019

             precision    recall  f1-score   support

          0       0.63      0.67      0.65    124185
          1       0.39      0.35      0.37     73843

avg / total       0.54      0.55      0.55    198028

             precision    recall  f1-score   support

          0       0.62      0.56      0.59    164393
          1       0.38      0.44      0.41     99645

avg / total       0.53      0.52      0.52    264038

             precision    recall  f1-score   support

          0       0.64      0.66      0.65    415477
          1       0.39      0.36 

AttributeError: 'LogisticRegressionCV' object has no attribute 'grid_scores_'

## Cross Validate SGD

In [ ]:
param_grid = [
  { 'alpha': [1e-3,5*1e-3,1e-2], 'l1_ratio': [3*1e-2,1e-3,6*1e-4], 'n_iter': [2,3]
  }
    ]

In [37]:
start_time = time.time()

param_grid = [
  { 'alpha': [1e-3,5*1e-3,1e-2], 'l1_ratio': [3*1e-2,1e-3,6*1e-4], 'n_iter': [2,3]
  }
    ]
sgd = SGDClassifier(loss='modified_huber', penalty='elasticnet', 
             fit_intercept=True,  shuffle=True, 
                    n_jobs=3,learning_rate='optimal', power_t =2, eta0 =5,
                    class_weight='balanced', average=40)

clf =GridSearchCV(sgd, param_grid, scoring='f1', fit_params=None, n_jobs=4, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf.fit(X_train,y_train)



predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['class'] = clf.predict(X_test)
predicted_proba[gt] =  y_test
predicted_proba.sort_values(by = 'prob',ascending = False, inplace = True)

cut_percentages = [10,20,30,40]
#converted = (predicted_proba['class']==1)
#me fijo todos los users que van a convertir en el futuro


start = 0
converted_dict = {}
ind_segments = []
for i in cut_percentages:
    cut_size = int(len(predicted_proba)*i/100.0)    
    bool_array = (~np.ones(len(predicted_proba), dtype=bool))
    bool_array[start: start + cut_size] = True
    ind_segments += [bool_array]
    start = start + cut_size
#    subtable = 
    #converted_dict[str(i)] = predicted_proba.loc[ind_segments[-1],'ref_hash']
    
    print(classification_report(predicted_proba.loc[ind_segments[-1],gt].values,
                            predicted_proba.loc[ind_segments[-1],'class'].values ))
    
    
print(classification_report(predicted_proba.loc[:,gt].values,
                            predicted_proba.loc[:,'class'].values ))
    
    
cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1
#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

all_time =   time.time() - start_time

#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

print('This cell took %s seconds to run' % all_time)


Fitting 3 folds for each of 18 candidates, totalling 54 fits


/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


[CV] n_iter=2, alpha=0.001, l1_ratio=0.03 ............................


/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


[CV] n_iter=2, alpha=0.001, l1_ratio=0.03 ............................


...

[CV]  n_iter=2, alpha=0.01, l1_ratio=0.0006000000000000001, score=0.122741 -   3.5s
[CV] n_iter=3, alpha=0.01, l1_ratio=0.0006000000000000001 ............


/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


[CV]  n_iter=3, alpha=0.01, l1_ratio=0.0006000000000000001, score=0.132433 -   3.7s
[CV] n_iter=3, alpha=0.01, l1_ratio=0.0006000000000000001 ............
[CV]  n_iter=3, alpha=0.01, l1_ratio=0.0006000000000000001, score=0.135111 -   3.4s
[CV]  n_iter=3, alpha=0.01, l1_ratio=0.0006000000000000001, score=0.135908 -   3.1s


[Parallel(n_jobs=4)]: Done  54 out of  54 | elapsed:  1.7min finished


             precision    recall  f1-score   support

          0       1.00      0.31      0.47    355992
          1       0.06      0.99      0.11     15152

avg / total       0.96      0.33      0.45    371144

This cell took 108.56432461738586 seconds to run


In [ ]:
threshold = 0.7
ind = predicted_proba['prob']>threshold
print(classification_report(predicted_proba.loc[ind,'y'].values,predicted_proba.loc[ind,'class'].values ))

In [38]:
clf.best_estimator_

SGDClassifier(alpha=0.005, average=40, class_weight='balanced', epsilon=0.1,
       eta0=5, fit_intercept=True, l1_ratio=0.0006000000000000001,
       learning_rate='optimal', loss='modified_huber', n_iter=3, n_jobs=3,
       penalty='elasticnet', power_t=2, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [39]:
coln=1
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())

n_iter
2.0    0.125741
3.0    0.132943
Name: mean_score, dtype: float64
n_iter
2.0    0.002609
3.0    0.002703
Name: mean_score, dtype: float64


### Results
* l1_ratio = cuanto mas chico mejor con lo cual la perdida l2 parece ser mejor
* alpha = 1e-3 es suficiente pues casi no afecta el score
* power_t = muy variado, no parece haber correlacion entre el tamanyo y el avg, mean_score
* eta0 = no afecta mucho pero parece ser que con ser >1 ya esta
* power_t = parecen ser practicamente iguales
* n_iter = cuanto mas iter NO mejora el score

In [51]:
len(predicted_proba)

371161

## Cross Validate NN

In [35]:
start_time = time.time()

X_train,X_test,y_train ,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

param_grid = { 'alpha': [1e-1,5*1e-2,1e-2],
              'hidden_layer_sizes':[(50,),(15,5)],
              'learning_rate': ['adaptive',"invscaling"],
              "algorithm": ['adam'],'momentum': [1e-2, 1e-1, 0.5],
  'power_t': [1e-3, 5*1e-4, 1e-5], 'activation':['logistic','relu']
 }

mlp = MLPClassifier(shuffle=True, 
                 verbose=True)

clf =GridSearchCV(mlp, param_grid, scoring='f1', fit_params=None, n_jobs=4, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf.fit(X_train,y_train)

predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['gt'] = y_test
predicted_proba['class'] = clf.predict(X_test)

threshold = 0.7
ind = predicted_proba['prob']>threshold

all_time =   time.time() - start_time

print(classification_report(predicted_proba.loc[ind,'gt'].values,predicted_proba.loc[ind,'class'].values ))

cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1
#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

Fitting 3 folds for each of 144 candidates, totalling 432 fits
[CV] algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive 
[CV] algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive 
[CV] algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive 
[CV] algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.0005, alpha=0.1, learning_rate=adaptive 
Iteration 1, loss = 0.20405210
Iteration 1, loss = 0.20380843
Iteration 1, loss = 0.20378706
Iteration 1, loss = 0.20405210


/home/teo/Repositories/scikit-learn/sklearn/metrics/classification.py:1097: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.0005, alpha=0.1, learning_rate=adaptive, score=0.000000 -   1.5s
[CV] algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.0005, alpha=0.1, learning_rate=adaptive 


/home/teo/Repositories/scikit-learn/sklearn/metrics/classification.py:1097: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive, score=0.000000 -   1.7s


/home/teo/Repositories/scikit-learn/sklearn/metrics/classification.py:1097: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive, score=0.000000 -   2.1s


/home/teo/Repositories/scikit-learn/sklearn/metrics/classification.py:1097: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive, score=0.000000 -   1.9s


KeyboardInterrupt: 

In [1]:
clf.best_estimator_

NameError: name 'clf' is not defined

In [52]:
# compare best parameters to tune
coln=1
cv_result.groupby(cv_result.columns[coln])['mean_score'].mean()
cv_result.groupby(cv_result.columns[coln])['mean_score'].std()

alpha
0.00     0.000007
0.01     0.000007
0.10     0.000007
0.50     0.000007
1.00     0.000007
10.00    0.000007
Name: mean_score, dtype: float64

### Results
* activation = logistic es 10% mejor
* alpha = 1e-2 el mejor 
* power_t = cuanto mas chico mejor, 1e-3 por lo menos
* hidden_layer_size = menos layers es mejor..?
* power_t = parecen ser practicamente iguales
* n_iter = cuanto mas iter NO mejora el score

## Bernoulli RBM features selection & Logit crossval

In [127]:
start_time = time.time()

## obs. este metodo es especial y asume que todos los valores son True/False o que \in [0,1]
# luego tengo que pensar en como tomar los features nuevamente.
# tampoco 'fittea' en la forma tradicional. Sino que se le puede tomar al y como un feature mas y esta red
# va 'modificando' todos los valores del X (minimizando la entropia) para dar un output. Luego corriendo 
# clf.gibbs(X_test) con el y_test como feature tmb, nos transforma la data para ver el output como la 'prediccion'
# obviamente no tiene probabilidades


#X = X_train[X_cols].values
#y = X_train['ground_truth'].values

df = X_train.drop(X_train[X_train[X_train.columns[0:3]].\
                                   sum(axis=1)==0].index)
df = df[X_cols + ['ground_truth']]

for col in X_train.columns[0:3]:
    df[col] = df[col]*1.0/df[df.columns[0:3]].sum(axis=1)

df[df.columns[3]] =  df[df.columns[3]]/df[df.columns[3]].max()
df[df.columns[4]] =  df[df.columns[4]]/df[df.columns[4]].max()

X = df[df.columns[:-1]].values
y = df['ground_truth'].values


param_grid = {'rbm__n_components': [256, 128,46,10],
   'rbm__n_iter':[15,10,5], 'rbm__learning_rate': [1e-4,1e-3,1e-2,1e-1,5*1e-3,5*1e-2,5*1e-1],
  'rbm__batch_size': [10e4,3*10e3, 1e3, 300],"logistic__C": [1.0, 10.0, 100.0] 
 }

rbm = BernoulliRBM(verbose=True)
logistic = LogisticRegression()
classifier = Pipeline([("rbm", rbm), ("logistic", logistic)])

X_train,X_test,y_train ,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

clf =GridSearchCV(classifier, param_grid, scoring='f1', fit_params=None, n_jobs=4, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf.fit(X_train,y_train)

#este metodo NO tiene predicted proba, lo que hacemos es recorrer 
#predicted_labels = rbm.gibbs(X_test)[:,-1]
#real_labels = X_test[:,-1]
#print(classification_report(real_labels,predicted_labels ))


predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['gt'] = y_test
predicted_proba['class'] = clf.predict(X_test)

threshold = 0.7
ind = predicted_proba['prob']>threshold
print(classification_report(predicted_proba.loc[ind,'gt'].values,predicted_proba.loc[ind,'class'].values ))
all_time =   time.time() - start_time

finished = True

cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1
    
print('This cell took %s seconds to run' % all_time)


KeyboardInterrupt: 

In [ ]:
coln=1
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())

## ADaboost CV

In [41]:
rf_param_grid = {'n_estimators': [10, 30, 100, 300, 1000]}
boost_param_grid = {'n_estimators': [10, 30, 100, 300, 1000],
                      'max_depth': [2, 3, 4, 5],
                      'min_samples_leaf': [1, 2, 3]}
ada_param_grid = {'n_estimators': [10, 30, 100, 300, 1000],
                   'learning_rate': [0.1, 0.3, 1.0, 3.0]}

X_train,X_test,y_train ,y_test = train_test_split(X,y, test_size=0.2, random_state=42)
#rf_est = RandomForestClassifier()
#rf_gs_cv = GridSearchCV(rf_est, rf_param_grid).fit(X_train, Y_train)
#print(rf_gs_cv.best_score_, rf_gs_cv.best_params_)
print('\n')
 
#boost_est = GradientBoostingClassifier()
#boost_gs_cv = GridSearchCV(boost_est, boost_param_grid).fit(X_train, y_train)
#print(boost_gs_cv.best_score_, boost_gs_cv.best_params_)
print('\n')
 
ada_est = AdaBoostClassifier()
ada_gs_cv = GridSearchCV(ada_est, ada_param_grid).fit(X_train, y_train)
print(ada_gs_cv.best_score_, ada_gs_cv.best_params_)
print('\n')

KeyboardInterrupt: 

In [42]:
start_time = time.time()


param_grid = {'n_estimators': [200,600,60,45,30,15],
 'learning_rate': [1e-4,1e-3,1e-2,1e-1,5*1e-3,5*1e-2,5*1e-1]
 }

aboost = AdaBoostClassifier(algorithm='SAMME.R')

clf =GridSearchCV(aboost, param_grid, scoring='f1', fit_params=None, n_jobs=4, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf.fit(X_train,y_train)

predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['gt'] = y_test
predicted_proba['class'] = clf.predict(X_test)

threshold = 0.7
ind = predicted_proba['prob']>threshold
print(classification_report(predicted_proba.loc[ind,'gt'].values,predicted_proba.loc[ind,'class'].values ))
all_time =   time.time() - start_time

print('This cell took %s seconds to run' % all_time)

finished = True
#este flag es basicamente para chequear 

cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1

finished = True

Fitting 3 folds for each of 42 candidates, totalling 126 fits
[CV] n_estimators=200, learning_rate=0.0001 ..........................
[CV] n_estimators=200, learning_rate=0.0001 ..........................
[CV] n_estimators=200, learning_rate=0.0001 ..........................
[CV] n_estimators=600, learning_rate=0.0001 ..........................
[CV] n_estimators=600, learning_rate=0.0001 ..........................


KeyboardInterrupt: 

In [23]:
coln=3
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())


criterion
gini    0.802919
Name: mean_score, dtype: float64
criterion
gini    0.000598
Name: mean_score, dtype: float64


## CrossV SVC

In [ ]:
start_time = time.time()

param_grid = [{'kernel': ['rbf'], 'gamma': list(10.0 ** np.arange(-2, 3)),
                     'C': list(10.0 ** np.arange(0, 4))},
                    {'kernel': ['poly'], 'C': list(10.0 ** np.arange(0, 4)), 'degree'[2,3,4]}]

svc = SVC(shuffle=True, probability=True,decision_function_shape = 'ovr',
           verbose=True, class_weight='balanced'
          )

clf =GridSearchCV(svc, param_grid, scoring='f1', fit_params=None, n_jobs=4, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf.fit(X_train,y_train)

predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['gt'] = y_test
predicted_proba['class'] = clf.predict(X_test)

threshold = 0.7
ind = predicted_proba['prob']>threshold

all_time =   time.time() - start_time

print(classification_report(predicted_proba.loc[ind,'gt'].values,predicted_proba.loc[ind,'class'].values ))

cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1
#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

In [ ]:
coln=3
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())


In [214]:
#tomo tiempo para saber cuanto tarda fitear el estimador
start_time = time.time()

#model = model.fit(X,y,sample_weight=W)

W = np.array([10 if i == 1 or i ==2  else 1 for i in y_mini])
gradboost.fit(X_mini,y_mini, sample_weight=W)

elapsed_time =   time.time() - start_time
print('GradientBoosting took %s seconds to run' % elapsed_time)

#validated =  cross_val_score(gradboost,X,y,cv=5, scoring = "f1_weighted")


all_time =   time.time() - start_time
print('This cell took %s seconds to run' % all_time)

GradientBoosting took 117.26272392272949 seconds to run
This cell took 117.26284193992615 seconds to run


## Timings
* 5s con  10 n_estimadores, 15 max_depth y X.sample(0.2)
* 15s con 10 n_estimadores, 15 max_depth y X.sample(0.2)
* 47s con 10 n_estimadores, 15 max_depth y X.sample(0.4)
* 35s con 10 n_estimadores, 15 max_depth y X.sample(0.4)
* 117 con 20 n_estimadores, 20 max_depth y X.sample(0.4)

In [31]:
X_train['ground_truth'].value_counts()

0    128153
1      7064
Name: ground_truth, dtype: int64

## Cross Validate - Random Forest

In [ ]:
param_grid = {'criterion': ['gini','entropy'], 'n_estimators': [30,50,60],
  'max_features': ["auto", "log2", "None"], "bootstrap": [ False],
    "min_samples_leaf": np.append(np.random.randint(1,11,3),[1]),'max_depth':[3,10, 15], 
              "min_samples_leaf": [1, 3, 10], "class_weight": ["None",'balanced']
              }

In [36]:
start_time = time.time()

param_grid = {'criterion': ['gini'], 'n_estimators': [15,30,50],
  'max_features': ["auto", "log2"], "bootstrap": [ False,True],
    "min_samples_leaf": np.append(np.random.randint(1,15,3),[1]),'max_depth':[3,10, 15], 
              "min_samples_leaf": [1, 4, 8], "class_weight": ['balanced']
              }

rforest  = RandomForestClassifier( )

clf =GridSearchCV(rforest, param_grid, scoring='f1', fit_params=None, n_jobs=8, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

#how many parameters to randomly search for
#n_iter_search = 45

#random_search = RandomizedSearchCV(rforest, param_distributions=param_grid,
                                 #  n_iter=n_iter_search, n_jobs =8, verbose=3)
#random_search.fit(X_train,y_train)
#elapsed_time =  time.time() - start_time
#print('Random Search took %s seconds to run' % elapsed_time)

clf.fit(X_train,y_train)

all_time =   time.time() - start_time 
print('Grid Search took %s seconds to run' % (all_time - elapsed_time))

predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_train)[:,1]
predicted_proba['class'] = clf.predict(X_train)
predicted_proba['ref_hash'] =  train_table.index.values
predicted_proba['y'] =  y_train
predicted_proba.sort_values(by = 'prob',ascending = False, inplace = True)

cut_percentages = [10,20,30,40]
#converted = (predicted_proba['class']==1)
#me fijo todos los users que van a convertir en el futuro
future_conversions = test_table[test_table['y']>0]['ref_hash'].values

start = 0
converted_dict = {}
ind_segments = []
for i in cut_percentages:
    cut_size = int(len(predicted_proba)*i/100.0)    
    bool_array = (~np.ones(len(predicted_proba), dtype=bool))
    bool_array[start: start + cut_size] = True
    ind_segments += [bool_array]
    start = start + cut_size
#    subtable = 
    converted_dict[str(i)] = predicted_proba.loc[ind_segments[-1],'ref_hash']
    
    print(classification_report(predicted_proba.loc[ind_segments[-1],'y'].values,
                            predicted_proba.loc[ind_segments[-1],'class'].values ))
    
print("Rates for %s app %s " % (appid_name[application_id], application_id))   

for i in cut_percentages:
    subtable = converted_dict[str(i)]
    
    print("{0} %% group rate: {1} size: {2}".format(str(i), subtable.isin(future_conversions).sum()*1.0/len(subtable), 
                                       len(subtable))) 
  

cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1
#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

all_time =   time.time() - start_time

#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

print('This cell took %s seconds to run' % all_time)


Fitting 3 folds for each of 108 candidates, totalling 324 fits


/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


[CV] class_weight=balanced, criterion=gini, max_features=auto, n_estimators=15, min_samples_leaf=1, bootstrap=False, max_depth=3 


/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


[CV] class_weight=balanced, criterion=gini, max_features=auto, n_estimators=15, min_samples_leaf=1, bootstrap=False, max_depth=3 


...

[CV]  class_weight=balanced, criterion=gini, max_features=log2, n_estimators=30, min_samples_leaf=8, bootstrap=True, max_depth=15, score=0.007491 -   9.1s
[CV] class_weight=balanced, criterion=gini, max_features=log2, n_estimators=50, min_samples_leaf=8, bootstrap=True, max_depth=15 


/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


[CV]  class_weight=balanced, criterion=gini, max_features=log2, n_estimators=30, min_samples_leaf=8, bootstrap=True, max_depth=15, score=0.000000 -   7.5s
[CV] class_weight=balanced, criterion=gini, max_features=log2, n_estimators=50, min_samples_leaf=8, bootstrap=True, max_depth=15 
[CV]  class_weight=balanced, criterion=gini, max_features=log2, n_estimators=30, min_samples_leaf=8, bootstrap=True, max_depth=15, score=0.000000 -   9.2s
[CV]  class_weight=balanced, criterion=gini, max_features=log2, n_estimators=50, min_samples_leaf=8, bootstrap=True, max_depth=15, score=0.031250 -  10.5s
[CV]  class_weight=balanced, criterion=gini, max_features=log2, n_estimators=50, min_samples_leaf=8, bootstrap=True, max_depth=15, score=0.000000 -  17.3s
[CV]  class_weight=balanced, criterion=gini, max_features=log2, n_estimators=50, min_samples_leaf=8, bootstrap=True, max_depth=15, score=0.000000 -  15.6s


[Parallel(n_jobs=8)]: Done 324 out of 324 | elapsed: 15.4min finished


Grid Search took 936.5473160743713 seconds to run
             precision    recall  f1-score   support

          0       1.00      0.99      1.00     35404
          1       0.06      0.86      0.11        22

avg / total       1.00      0.99      1.00     35426

             precision    recall  f1-score   support

          0       1.00      1.00      1.00     70852

avg / total       1.00      1.00      1.00     70852

             precision    recall  f1-score   support

          0       1.00      1.00      1.00    106278

avg / total       1.00      1.00      1.00    106278

             precision    recall  f1-score   support

          0       1.00      1.00      1.00    141705

avg / total       1.00      1.00      1.00    141705

Rates for privalia app 460 
10 %% group rate: 0.013351775532095071 size: 35426
20 %% group rate: 0.026138993959238976 size: 70852
30 %% group rate: 0.0843071943393741 size: 106278
40 %% group rate: 0.09898027592533785 size: 141705
This cell took 939

In [16]:
X_train.shape

(94414, 632)

In [37]:
clf.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=15, max_features='auto',
            max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [38]:
np.bincount(y_train),np.bincount(y_test)

(array([354241,     22]), array([415166,  30911]))

In [13]:
np.bincount(y_train),np.bincount(y_test)

(array([338477,  16103]), array([356009,  15152]))

In [17]:
a = classification_report(predicted_proba.loc[ind,'y'].values,predicted_proba.loc[ind,'class'].values )

In [34]:
cut_up = 0.8
cut_down = 0.6

ind_up = predicted_proba['prob']>cut_up
ind_middle = (predicted_proba['prob']>cut_down) & (predicted_proba['prob']>cut_down )
ind_down = (predicted_proba['prob']<cut_down)

converted = predicted_proba['class']==1

print(classification_report(predicted_proba.loc[ind_up,'y'].values,predicted_proba.loc[ind_up,'class'].values ))

print(classification_report(predicted_proba.loc[ind_middle,'y'].values,predicted_proba.loc[ind_middle,'class'].values ))

print(classification_report(predicted_proba.loc[ind_down,'y'].values,predicted_proba.loc[ind_down,'class'].values ))

print("High: rate: {0} size: {1}".format( (converted & ind_up).sum()/ind_up.sum(), ind_up.sum()))
print("Middle: rate: {0} size: {1}".format((converted & ind_down).sum()/ind_middle.sum(), ind_middle.sum()))
print("Low: rate: {0} size: {1}".format((converted & ind_down).sum()/ind_down.sum(), ind_down.sum()))

             precision    recall  f1-score   support

          0       1.00      0.98      0.99    307009
          1       0.53      0.90      0.67      6756

avg / total       0.99      0.98      0.98    313765

             precision    recall  f1-score   support

          0       0.99      0.95      0.97    341967
          1       0.40      0.84      0.54     12785

avg / total       0.97      0.95      0.96    354752

             precision    recall  f1-score   support

          0       0.89      0.55      0.68     14042
          1       0.18      0.58      0.27      2367

avg / total       0.78      0.56      0.62     16409

High: rate: 0.03629467913884595 size: 313765
Middle: rate: 0.021598186902399424 size: 354752
Low: rate: 0.46693887500761777 size: 16409


In [33]:
threshold = 0.5
ind = predicted_proba['prob']>threshold

print(classification_report(predicted_proba.loc[ind,'y'].values,predicted_proba.loc[ind,'class'].values ))


             precision    recall  f1-score   support

          0       0.99      0.94      0.96    356009
          1       0.35      0.80      0.49     15152

avg / total       0.96      0.93      0.94    371161



In [68]:
coln=2
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())

min_samples_leaf
5.0     0.427722
10.0    0.380114
Name: mean_score, dtype: float64
min_samples_leaf
5.0     0.012958
10.0    0.005532
Name: mean_score, dtype: float64


### Results
* NO escalar (normalizar, restar la media  dividir por la std, etc) los resultados pues REVIENTAN los scores.
* bootstrap  = False es 5% mejor
* min_samples_leaf = mas chico es claramente mejor, pero tmb aumenta el overfitting lo cual me hace caer mucho el valor del recall en el test_set. Sin embargo es un parametro muy sensible en la precision. Resta evaluar asi el tradeoff entre la precision y el volumen de users al cual queremos llegar.
* n_estimators = aumentar mas de 30 no tendria mucho sentido
* citerion = entropy o gini no cambia. gini podria ser mejor entonces pues entropy usa logs de los valores lo cual es mas computacionalmente costoso
* max_features = no afecta al score. con auto esta bien
* max_depth =  mas es mejor. intentaria probar con >15

## Cross Validate ExtraTreeClassifier

In [94]:
predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_train)[:,1]
predicted_proba['class'] = clf.predict(X_train)
predicted_proba['ref_hash'] =  train_table.index.values
predicted_proba['y'] =  y_train
predicted_proba.sort_values(by = 'prob',ascending = False, inplace = True)

cut_percentages = [10,20,30,40]
#converted = (predicted_proba['class']==1)
#me fijo todos los users que van a convertir en el futuro
future_conversions = test_table[test_table['y']>0]['ref_hash'].values


start = 0
converted_dict = {}
ind_segments = []
for i in cut_percentages:
    cut_size = int(len(predicted_proba)*i/100.0)    
    bool_array = (~np.ones(len(predicted_proba), dtype=bool))
    bool_array[start: start + cut_size] = True
    ind_segments += [bool_array]
    start = start + cut_size
#    subtable = 
    converted_dict[str(i)] = predicted_proba.loc[ind_segments[-1],'ref_hash']
    
    print(classification_report(predicted_proba.loc[ind_segments[-1],'y'].values,
                            predicted_proba.loc[ind_segments[-1],'class'].values ))
    
print("Rates for %s app %s " % (appid_name[application_id], application_id))   
for i in cut_percentages:
    subtable = converted_dict[str(i)]
    
    print("{0} %% group rate: {1} size: {2}".format(str(i), subtable.isin(future_conversions).sum()*1.0/len(subtable), 
                                       len(subtable))) 


/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/metrics/classification.py:1117: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.00      0.00      0.00     26720
          1       0.25      1.00      0.40      8738

avg / total       0.06      0.25      0.10     35458

             precision    recall  f1-score   support

          0       0.97      0.43      0.60     66920
          1       0.07      0.76      0.14      3996

avg / total       0.92      0.45      0.57     70916

             precision    recall  f1-score   support

          0       0.99      1.00      0.99    105255
          1       0.00      0.00      0.00      1119

avg / total       0.98      0.99      0.98    106374

             precision    recall  f1-score   support

          0       0.98      1.00      0.99    139582
          1       0.00      0.00      0.00      2250

avg / total       0.97      0.98      0.98    141832

Rates for dafiti app 189 
10 %% group rate: 0.10429240227875233 size: 35458
20 %% group rate: 0.024479666083817474 size: 70916
30 %% group r

In [100]:
coln = 2
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())

alpha
0.00     0.000000
0.01     0.242260
0.10     0.242266
0.50     0.242220
1.00     0.242200
10.00    0.241269
Name: mean_score, dtype: float64
alpha
0.00     0.000000
0.01     0.017568
0.10     0.017598
0.50     0.017578
1.00     0.017570
10.00    0.017780
Name: mean_score, dtype: float64


## Cross validate MNB

In [72]:
start_time = time.time()

# pero tenemos que transformar a y en un vector canonico indicando en 1 en la posicion correspondiente a su categoria
#y =  label_binarize(X_train['ground_truth'].values.astype(int),
#                    classes=list(set(X_train['ground_truth'].values.astype(int))))

perc = 0.3
mini_ind = np.random.choice(X_train.shape[0],int(perc*X_train.shape[0]),replace=False)
#X_mini =  X[mini_ind,:]
#y_mini = y[mini_ind]


clf = MultinomialNB(
        )

validated = []
# Only take the first fold.
n_folds = 4
for i in range(n_folds):
    # Break up the dataset into non-overlapping training (75%) and testing
    # (25%) sets.
    skf = StratifiedKFold(y_train, n_folds=n_folds)
    train_index, test_index = next(iter(skf))

    X_traincv = X_train[train_index]
    y_traincv = y_train[train_index]

    n_classes = len(np.unique(y_traincv))

    clf.fit(X_traincv,y_traincv)
    predictions = clf.predict(X_traincv)
        
    expected = y_traincv

    print(classification_report(expected, predictions))


predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['y'] = y_test
predicted_proba['class'] = clf.predict(X_test)


#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

all_time =   time.time() - start_time
print('This cell took %s seconds to run' % all_time)


             precision    recall  f1-score   support

          0       0.99      0.85      0.91     70323
          1       0.12      0.73      0.20      1932

avg / total       0.97      0.84      0.89     72255

             precision    recall  f1-score   support

          0       0.99      0.85      0.91     70323
          1       0.12      0.73      0.20      1932

avg / total       0.97      0.84      0.89     72255

             precision    recall  f1-score   support

          0       0.99      0.85      0.91     70323
          1       0.12      0.73      0.20      1932

avg / total       0.97      0.84      0.89     72255

             precision    recall  f1-score   support

          0       0.99      0.85      0.91     70323
          1       0.12      0.73      0.20      1932

avg / total       0.97      0.84      0.89     72255

This cell took 1.06442093849 seconds to run


In [76]:
threshold = 0.99
ind = predicted_proba['prob']>threshold
print(classification_report(predicted_proba.loc[ind,'y'].values,predicted_proba.loc[ind,'class'].values ))


             precision    recall  f1-score   support

          0       0.99      0.87      0.93    185744
          1       0.15      0.73      0.26      5906

avg / total       0.96      0.87      0.91    191650



In [60]:
#tomo tiempo para saber cuanto tarda fitear el estimador
start_time = time.time()

svc2.fit(X_mini,y_mini)
elapsed_time =   time.time() - start_time
print('SVC took %s seconds to run' % elapsed_time)

all_time =   time.time() - start_time
print('This cell took %s seconds to run' % all_time)

SVC took 362.8217749595642 seconds to run
This cell took 362.8219118118286 seconds to run


## Timings
* 40k samples : 360s
* 

In [75]:
start_time =  time.time()
lr.fit(X_train,y_train)
elapsed_time =   time.time() - start_time
print('Logit took %s seconds to run' % elapsed_time)

Logit took 1073.0756621360779 seconds to run


In [ ]:
#tomo tiempo para saber cuanto tarda fitear el estimador
start_time = time.time()

lr.fit(X_train,y_train)
elapsed_time =   time.time() - start_time
print('Logit took %s seconds to run' % elapsed_time)

svc.fit(X_train,y_train)
elapsed_time =   time.time() - start_time
print('Linear SVC took %s seconds to run' % elapsed_time)

all_time =   time.time() - start_time
print('This cell took %s seconds to run' % all_time)

Logit took 89.15975689888 seconds to run
Linear SVC took 804.7178750038147 seconds to run

In [ ]:
y.value_counts

# Todo
* evaluate hit_rate and 
* tune adaboost, bernoulliRBM
* xgboost
* libffm
* SVC muy lento.. speed up in AWS
